¿Cómo importar la biblioteca?
==

In [2]:
import athena2pyspark as ath # se demora un poco porque esta creado el SparkContext

Hacer una query y que te retorne un dataframe
==

Podemos usar los recursos de athena para obtener querys basado en sql, esto nos permite hacer querys gigantes de forma server-less

In [4]:
s3_output = "s3://leonardo.exalitica.com/boto3/query_1/"
df = ath.run_query(query = "select * from baul_2 limit 1000", database = "ljofre", s3_output = s3_output)

Execution ID: 8da33860-e60e-42a9-9e83-79120a4a7c95


In [13]:
df.toPandas().head(100)

,party_id,corr,cv_us,cv_u4s,cv_u12s,m_us,m_u4s,m_u12s,cp_us,cp_u4s,cp_u12s,rec,tu4s,tu12s,n_key
0,100183795,1875,0,1,1,0.0000,467.2269,467.2269,0.0,4.000,4.000,23.0,30.0,90.000000,2016_1
1,100189452,7820,0,0,2,0.0000,0.0000,2268.0672,0.0,0.000,5.000,36.0,30.0,50.000000,2016_1
2,100384605,3472,0,1,3,0.0000,2335.2941,9475.6302,0.0,1.000,4.000,19.0,30.0,30.500000,2016_1
3,100390335,20113,0,1,1,0.0000,2436.1345,2436.1345,0.0,1.000,1.000,21.0,30.0,90.000000,2016_1
4,100581556,31907,0,1,1,0.0000,2512.6050,2512.6050,0.0,1.000,1.000,25.0,30.0,90.000000,2016_1
5,100588408,265,0,0,1,0.0000,0.0000,1612.6050,0.0,0.000,0.240,75.0,30.0,90.000000,2016_1
6,100783295,3337,0,1,2,0.0000,586.5546,1189.9159,0.0,2.000,4.000,23.0,30.0,60.000000,2016_1
7,100794944,1426,0,0,2,0.0000,0.0000,3233.6135,0.0,0.000,2.000,59.0,30.0,22.000000,2016_1
8,100981604,2637,0,1,4,0.0000,11138.6554,51860.5039,0.0,0.942,4.185,20.0,30.0,15.333333,2016_1
9,100991407,33684,0,1,1,0.0000,2008.4034,2008.4034,0.0,1.000,1.000,18.0,30.0,90.000000,2016_1


Generar el "create table" a partir del dataframe ya creado
==

Es util registrar este dataframe dentro de un catálogo para poder seguir haciendo consultas dentro de athena

In [8]:
s3_input = "s3://leonardo.exalitica.com/boto3/query_1/8da33860-e60e-42a9-9e83-79120a4a7c95.csv"
create_database, create_table = ath.get_ddl(df=df,
                                            database="ljofre",
                                            table="nueva_tabla",
                                            s3_input=s3_input)

In [11]:
print(create_database)

CREATE DATABASE IF NOT EXISTS ljofre;


In [12]:
print(create_table)

CREATE EXTERNAL TABLE IF NOT EXISTS ljofre.nueva_tabla ('party_id' string,'corr' string,'cv_us' string,'cv_u4s' string,'cv_u12s' string,'m_us' string,'m_u4s' string,'m_u12s' string,'cp_us' string,'cp_u4s' string,'cp_u12s' string,'rec' string,'tu4s' string,'tu12s' string,'n_key' string)
     ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
     WITH SERDEPROPERTIES (
     'serialization.format' = '1'
     ) LOCATION 's3://leonardo.exalitica.com/boto3/query_1/8da33860-e60e-42a9-9e83-79120a4a7c95.csv'
     TBLPROPERTIES ('has_encrypted_data'='false');


Ejecutar create table y dejar registrada la tabla en Athena
==